Note: some of the starting code was built upon notebooks developed for the FMA dataset, found at the FMA dataset github: https://github.com/mdeff/fma


In [1]:
import pandas as pd
import numpy as np
import seaborn as sns 
import matplotlib.pyplot as plt
import sklearn as skl
import sklearn.utils, sklearn.preprocessing, sklearn.decomposition, sklearn.svm

from sklearn import metrics
from sklearn.model_selection import KFold,GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.metrics import precision_score, recall_score, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC

pd.set_option('display.max_columns', None)

In [2]:
features = pd.read_csv("fma_metadata/features.csv", header=[0,1,2], index_col=0)
tracks = pd.read_csv("fma_metadata/tracks.csv", header = [0,1], index_col=0)

# echonest = pd.read_csv("fma_metadata/echonest.csv", header = [0,1,2], index_col=0)
# genres = pd.read_csv("fma_metadata/genres.csv", index_col=0)

In [16]:
features.shape, tracks.shape #, echonest.shape, genres.shape

((106574, 518), (106574, 52))

In [4]:
features.sample(10)

feature    chroma_cens                                                    \
statistics    kurtosis                                                     
number              01        02        03        04        05        06   
track_id                                                                   
74243        -0.415694 -0.938237  1.244834  2.671037 -1.106524 -1.126895   
92423        -0.202370 -0.466249 -0.502340 -0.284807 -0.067449  0.999438   
95518         0.344596 -0.591124 -0.183028 -0.790170 -0.211429 -0.227485   
129498       -0.546652 -0.591767 -0.482967 -0.359699 -0.623235 -0.101425   
71778        -0.825305 -0.804667 -0.113884 -0.846125 -0.415716 -0.274891   
138730       -1.071539 -0.908698 -0.950621 -0.402187 -1.261216 -1.118107   
115309       -0.041136  0.009145  1.000720 -0.641884 -0.879936 -0.922513   
13109         0.090021  0.505498 -0.229210  0.503387 -0.092599  0.208086   
97011         0.279314  0.790964  0.924833  0.992707  0.781030  0.378350   
115548       -0.615375 -0.682027 -0.725960 -0.774406 -0.668306 -0.649894   

feature                                                                 \
statistics                                                               
number            07        08        09        10        11        12   
track_id                                                                 
74243      -0.453268 -0.496512 -1.132859 -0.538025  0.780370 -0.619817   
92423      -0.173368 -0.632421 -0.254061  0.307117  0.227643  0.097975   
95518      -0.800464 -0.879815 -1.073921 -0.686551  0.340496 -0.296309   
129498      0.248867 -0.559550 -0.464049 -0.165753 -0.141037 -0.479195   
71778      -0.511958 -0.453637 -0.736990 -0.495133 -0.453425  0.850278   
138730     -0.826717  0.877722  0.395251 -0.608360 -0.846634 -0.711223   
115309      0.226089  0.959091  0.785826  1.659530 -0.153554 -0.126148   
13109       1.025497 -0.125757  0.973954  0.219448  0.174616 -0.166821   
97011       0.484987  0.867680  1.286739  0.829354  0.504685  1.869831   
115548     -0.798500 -0.269079 -0.241490 -0.931352 -0.864595 -1.053048   

feature                                                                 \
statistics       max                                                     
number            01        02        03        04        05        06   
track_id                                                                 
74243       0.689929  0.495869  0.634985  0.475988  0.641185  0.710024   
92423       0.702470  0.484800  0.632695  0.694081  0.471819  0.691382   
95518       0.642530  0.488507  0.615255  0.514711  0.687608  0.572221   
129498      0.615662  0.673756  0.669485  0.630911  0.690112  0.613335   
71778       0.656279  0.614439  0.537235  0.553158  0.709008  0.615577   
138730      0.677808  0.704631  0.634877  0.699841  0.524906  0.687212   
115309      0.694127  0.668759  0.696903  0.648474  0.681868  0.599346   
13109       0.608718  0.635907  0.663240  0.639454  0.648883  0.648204   
97011       0.587580  0.630134  0.624641  0.615254  0.616023  0.537470   
115548      0.671245  0.702069  0.630513  0.621174  0.559508  0.684963   

feature                                                                 \
statistics                                                               
number            07        08        09        10        11        12   
track_id                                                                 
74243       0.637014  0.715609  0.623677  0.502431  0.631337  0.526197   
92423       0.499300  0.677984  0.480552  0.494060  0.632992  0.474400   
95518       0.503738  0.674055  0.519174  0.524888  0.636455  0.617239   
129498      0.570575  0.648612  0.631956  0.656402  0.640902  0.668178   
71778       0.651055  0.629963  0.463077  0.608788  0.525465  0.603446   
138730      0.499213  0.499187  0.710505  0.637698  0.724959  0.523574   
115309      0.600360  0.657144  0.632959  0.626299  0.614053  0.682918   
13109       0.648671  0.626339  0.636828  0.676

In [5]:
tracks.sample(10)

album                                            \
         comments         date_created        date_released   
track_id                                                      
118165          0  2015-04-19 16:23:02  2015-04-19 00:00:00   
10189           0  2009-04-08 18:18:31  2007-05-26 00:00:00   
8406            0  2009-03-25 12:33:36  2003-02-13 00:00:00   
39726           0  2010-11-27 12:29:18  2010-11-27 00:00:00   
66355           0  2012-06-11 17:36:58  2012-06-06 00:00:00   
22056           0  2009-11-12 21:09:52  2009-10-26 00:00:00   
99496           0  2014-02-24 08:58:06  2014-02-18 00:00:00   
127211          0  2015-11-03 10:29:00  2015-05-01 00:00:00   
122149          0  2015-07-03 12:37:54  2015-07-03 00:00:00   
73004           0  2012-11-26 08:32:38  2012-11-26 00:00:00   

                                                    \
                         engineer favorites     id   
track_id                                             
118165                        NaN         0  18117   
10189                         NaN         2   2515   
8406                          NaN         1   2181   
39726                         NaN         0   7901   
66355     Diane "Kamikaze" Farris         0  11915   
22056                         NaN         0   4676   
99496                         NaN         1  16074   
127211                        NaN         2  19229   
122149                        NaN         0  18507   
73004                         NaN         0  12948   

                                                                     \
                                                information listens   
track_id                                                              
118165    <p>Celebrating 6 years of Section 27, we prese...   82634   
10189     <p>Debut release from Yes No Wave Music. A com...    5004   
8406                                                    NaN    5950   
39726     <p><span style="font-family:Arial, Helvetica, ...    3283   
66355     <p>This is '70s-influenced garage-punk at its ...    4856   
22056     <p>Learning Music Monthly is a subscription ba...    9507   
99496     <p><strong>Want high-resolution album art, lin...    6413   
127211    <p>Une bonne occasion de redécouvrir les sonor...   34431   
122149    <p><span style="font-family: 'Lucida Grande'; ...   16424   
73004     <p style="text-align: justify; margin-top: 0pt...    7285   

                              \
                    producer   
track_id                       
118165                   NaN   
10189           Wok The Rock   
8406                     NaN   
39726                    NaN   
66355     Evan "Funk" Davies   
22056                    NaN   
99496                    NaN   
127211                   NaN   
122149        Jaan Patterson   
73004                    NaN   

                                                             \
                                                       tags   
track_id                                                      
118165    ['compilation', 'electronic glitch idm experim...   
10189                                                    []   
8406                                                     []   
39726                                                    []   
66355                                                    []   
22056                                                    []   
99496                                                    []   
127211                                                   []   
122149    ['c hinz', 'william davison', 'alessandra cell...   
73004     ['experimental', 'electronica', 'electronic', ...   

                                                                           \
                                                      title tracks   type   
track_id                                                                    
118165                                       Sectioned v5.0     83  Album   
10189               

# EDA / Data Cleaning

### Questions:
- What are the top genres in this dataset? 
- What does the distribution of top genres look like? 
- How many "top genres" are NA?
- What to do with the huge class imbalance (counts differenct a factor of 100 - 1000)?

### Observations:
- The "tracks" dataset is organized with a top category (ie album, artist, set, track) and subcategories, so columns must be accessed in this way (see next cell as an example)
- Over HALF of the tracks do not have value in "genre_top" column

#### the full dataset has NaN for about half of the genre_top features

In [6]:
# Top Genres list for the entire set 
tracks.loc[:,('track','genre_top')].value_counts(dropna=False)

NaN                    56976
Rock                   14182
Experimental           10608
Electronic              9372
Hip-Hop                 3552
Folk                    2803
Pop                     2332
Instrumental            2079
International           1389
Classical               1230
Jazz                     571
Old-Time / Historic      554
Spoken                   423
Country                  194
Soul-RnB                 175
Blues                    110
Easy Listening            24
Name: (track, genre_top), dtype: int64

In [9]:
# Create a large subset that drops rows with NaN in the genre_top column
large = tracks[tracks['track','genre_top'].notna()]
large.loc[:,('track','genre_top')].value_counts(dropna=False)

Rock                   14182
Experimental           10608
Electronic              9372
Hip-Hop                 3552
Folk                    2803
Pop                     2332
Instrumental            2079
International           1389
Classical               1230
Jazz                     571
Old-Time / Historic      554
Spoken                   423
Country                  194
Soul-RnB                 175
Blues                    110
Easy Listening            24
Name: (track, genre_top), dtype: int64

#### Notice there's a huge class imbalance in the largest dataset. I trim it down below:

In [14]:
# create a large subset with only genres that have at least 1000 tracks 
over_1000 = ["Rock","Experimental","Electronic","Hip_Hop","Folk","Pop","Instrumental","International","Classical"]
large_b = large[large['track','genre_top'].isin(over_1000)]
large_b.loc[:,('track','genre_top')].value_counts(dropna=False)

Rock             14182
Experimental     10608
Electronic        9372
Folk              2803
Pop               2332
Instrumental      2079
International     1389
Classical         1230
Name: (track, genre_top), dtype: int64

#### Grouping features to be used with the classification models

In [20]:
# What are the major features in the "features" dataset?
feature_list = features.columns.levels[0].tolist()
feature_list

['chroma_cens',
 'chroma_cqt',
 'chroma_stft',
 'mfcc',
 'rmse',
 'spectral_bandwidth',
 'spectral_centroid',
 'spectral_contrast',
 'spectral_rolloff',
 'tonnetz',
 'zcr']

In [21]:
# Add a few feature COMBINATIONS (based on EDA done in the MVP)

# Top 2 features with best accuracy scores by SVC classification
feature_list.append(['mfcc','spectral_contrast'])
# Top 4 features with best accuracy scores by SVC classification
feature_list.append(['mfcc','spectral_contrast','chroma_cqt','chroma_stft'])
# All the features, EXCLUDING the 4 features with the worst accuracy scores by SVC classification
feature_list.append(['chroma_cens','chroma_cqt','chroma_stft','mfcc','spectral_centroid',
                     'spectral_contrast','tonnetz'])
# All the features!
feature_list.append(['chroma_cens','chroma_cqt','chroma_stft','mfcc','rmse','spectral_bandwidth','spectral_centroid',
                     'spectral_contrast','spectral_rolloff','tonnetz','zcr'])

# Indexes for the tables
indexes = ['chroma_cens','chroma_cqt','chroma_stft','mfcc','rmse','spectral_bandwidth','spectral_centroid',
           'spectral_contrast','spectral_rolloff','tonnetz','zcr',
           'mfcc/spectral_contrast',
           'mfcc/spectral_contrast/chroma_cqt/chroma_stft',
           'chroma_cens/chroma_cqt/chroma_stft/mfcc/spectral_centroid/spectral_contrast/tonnetz',
           'ALL']

In [22]:
feature_list


['chroma_cens',
 'chroma_cqt',
 'chroma_stft',
 'mfcc',
 'rmse',
 'spectral_bandwidth',
 'spectral_centroid',
 'spectral_contrast',
 'spectral_rolloff',
 'tonnetz',
 'zcr',
 ['mfcc', 'spectral_contrast'],
 ['mfcc', 'spectral_contrast', 'chroma_cqt', 'chroma_stft'],
 ['chroma_cens',
  'chroma_cqt',
  'chroma_stft',
  'mfcc',
  'spectral_centroid',
  'spectral_contrast',
  'tonnetz'],
 ['chroma_cens',
  'chroma_cqt',
  'chroma_stft',
  'mfcc',
  'rmse',
  'spectral_bandwidth',
  'spectral_centroid',
  'spectral_contrast',
  'spectral_rolloff',
  'tonnetz',
  'zcr']]

# Data Analysis


In [46]:
# for lack of a better method, create a mask, LARGE, that only has "top_genres" with at least 1000tracks
LARGE = ((tracks['track','genre_top'].notna()) & (tracks['track','genre_top'].isin(over_1000)))
LARGE

track_id
2         False
3         False
5         False
10         True
20        False
          ...  
155316     True
155317     True
155318     True
155319     True
155320    False
Name: (track, genre_top), Length: 106574, dtype: bool

In [48]:
# add first column that shows the number of dimensions for each feature (or collection of features)
dim_col = []
for feature in feature_list:
    X = features.loc[LARGE, (feature)]
    dim_col.append(X.shape[1])
accuracy_scores['dim']=dim_col

accuracy_scores

,dim
chroma_cens,84
chroma_cqt,84
chroma_stft,84
mfcc,140
rmse,7
spectral_bandwidth,7
spectral_centroid,7
spectral_contrast,49
spectral_rolloff,7
tonnetz,42


In [ ]:
recall_scores = pd.DataFrame(index=indexes)

# Add first column with the number of dimensions for each of the features
dim_col = []
for feature in feature_list:
    X = features.loc[large_b, (feature)]
    dim_col.append(X.shape[1])
recall_scores['dim']=dim_col


In [ ]:
f1_scores = pd.DataFrame(index=indexes)

# Add first column with the number of dimensions for each of the features
dim_col = []
for feature in feature_list:
    X = features.loc[large_b, (feature)]
    dim_col.append(X.shape[1])
f1_scores['dim']=dim_col


### Logistic Regression

In [ ]:
%%time

accuracy_col = []
recall_col = []
f1_col = []

for feature in feature_list:
    y = tracks.loc[large_b, ('track','genre_top')]
    X = features.loc[large_b, feature]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    scaler = StandardScaler(copy=False)
    scaler.fit_transform(X_train)
    scaler.transform(X_test)
    lr = LogisticRegression(max_iter=1000)
    lr.fit(X_train, y_train)
    y_pred = lr.predict(X_test)
    accuracy_col.append(metrics.accuracy_score(y_test, y_pred).round(3))
#     recall_col.append(metrics.recall_score(y_test, y_pred,average='macro').round(3))
#     f1_col.append(metrics.f1_score(y_test, y_pred,average='macro').round(3))

accuracy_scores['lr'] = accuracy_col
# recall_scores['lr'] = recall_col
# f1_scores['lr'] = f1_col

#### Weighted Logistic Regression

In [ ]:
%%time

accuracy_col = []
recall_col = []
f1_col = []

for feature in feature_list:
    y = tracks.loc[large_b, ('track','genre_top')]
    X = features.loc[large_b, feature]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    scaler = StandardScaler(copy=False)
    scaler.fit_transform(X_train)
    scaler.transform(X_test)
    lr_weighted = LogisticRegression(max_iter=1000,class_weight={'International' : 4, 'Classical' : 4, 
                                               'Instrumental':3,'Folk':2, 'Pop':2}, solver='liblinear')
    lr_weighted.fit(X_train, y_train)
    y_pred = lr_weighted.predict(X_test)
    accuracy_col.append(metrics.accuracy_score(y_test, y_pred).round(3))
#     recall_col.append(metrics.recall_score(y_test, y_pred,average='macro').round(3))
#     f1_col.append(metrics.f1_score(y_test, y_pred,average='macro').round(3))

accuracy_scores['weighted lr'] = accuracy_col
# recall_scores['weighted lr'] = recall_col
# f1_scores['weighted lr'] = f1_col

### K Nearest Neighbors

In [ ]:
%%time

accuracy_col = []
recall_col = []
f1_col = []

for feature in feature_list:
    y = tracks.loc[large_b, ('track','genre_top')]
    X = features.loc[large_b, (feature)]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    scaler = StandardScaler(copy=False)
    scaler.fit_transform(X_train)
    scaler.transform(X_test)
    knn = KNeighborsClassifier(n_neighbors=200)
    knn.fit(X_train, y_train)
    y_pred = knn.predict(X_test)
    accuracy_col.append(metrics.accuracy_score(y_test, y_pred).round(3))
#     recall_col.append(metrics.recall_score(y_test, y_pred,average='macro').round(3))
#     f1_col.append(metrics.f1_score(y_test, y_pred,average='macro').round(3))

accuracy_scores['knn'] = accuracy_col
# recall_scores['knn'] = recall_col
# f1_scores['knn'] = f1_col

### Decision Tree

In [ ]:
%%time

accuracy_col = []
recall_col = []
f1_col = []

for feature in feature_list:
    y = tracks.loc[large_b, ('track','genre_top')]
    X = features.loc[large_b, feature]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    scaler = StandardScaler(copy=False)
    scaler.fit_transform(X_train)
    scaler.transform(X_test)
    DT = DecisionTreeClassifier(max_depth=5)
    DT.fit(X_train, y_train)
    y_pred = DT.predict(X_test)
    accuracy_col.append(metrics.accuracy_score(y_test, y_pred).round(3))
#     recall_col.append(metrics.recall_score(y_test, y_pred,average='macro').round(3))
#     f1_col.append(metrics.f1_score(y_test, y_pred,average='macro').round(3))

accuracy_scores['dt'] = accuracy_col
# recall_scores['dt'] = recall_col
# f1_scores['dt'] = f1_col

### Random Forest

In [ ]:
%%time

accuracy_col = []
recall_col = []
f1_col = []

for feature in feature_list:
    y = tracks.loc[large_b, ('track','genre_top')]
    X = features.loc[large_b, feature]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    scaler = StandardScaler(copy=False)
    scaler.fit_transform(X_train)
    scaler.transform(X_test)
    RF = RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1)
    RF.fit(X_train, y_train)
    y_pred = RF.predict(X_test)
    accuracy_col.append(metrics.accuracy_score(y_test, y_pred).round(3))
#     recall_col.append(metrics.recall_score(y_test, y_pred,average='macro').round(3))
#     f1_col.append(metrics.f1_score(y_test, y_pred,average='macro').round(3))

accuracy_scores['rf'] = accuracy_col
# recall_scores['rf'] = recall_col
# f1_scores['rf'] = f1_col

### Ada Boost

In [ ]:
%%time

accuracy_col = []
recall_col = []
f1_col = []

for feature in feature_list:
    y = tracks.loc[large_b, ('track','genre_top')]
    X = features.loc[large_b, feature]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    scaler = StandardScaler(copy=False)
    scaler.fit_transform(X_train)
    scaler.transform(X_test)
    ada = AdaBoostClassifier(n_estimators=10)
    ada.fit(X_train, y_train)
    y_pred = ada.predict(X_test)
    accuracy_col.append(metrics.accuracy_score(y_test, y_pred).round(3))
#     recall_col.append(metrics.recall_score(y_test, y_pred,average='macro').round(3))
#     f1_col.append(metrics.f1_score(y_test, y_pred,average='macro').round(3))

accuracy_scores['ada'] = accuracy_col
# recall_scores['ada'] = recall_col
# f1_scores['ada'] = f1_col

### Naive Bayes

In [ ]:
%%time

accuracy_col = []
recall_col = []
f1_col = []

for feature in feature_list:
    y = tracks.loc[large_b, ('track','genre_top')]
    X = features.loc[large_b, feature]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    scaler = StandardScaler(copy=False)
    scaler.fit_transform(X_train)
    scaler.transform(X_test)
    nbc_base = GaussianNB()
    nbc_base.fit(X_train, y_train)
    y_pred = nbc_base.predict(X_test)
    accuracy_col.append(metrics.accuracy_score(y_test, y_pred).round(3))
#     recall_col.append(metrics.recall_score(y_test, y_pred,average='macro').round(3))
#     f1_col.append(metrics.f1_score(y_test, y_pred,average='macro').round(3))

accuracy_scores['nb'] = accuracy_col
# recall_scores['nb'] = recall_col
# f1_scores['nb'] = f1_col

### SVC Classifier

In [ ]:
%%time

accuracy_col = []
recall_col = []
f1_col = []

for feature in feature_list:
    y = tracks.loc[large_b, ('track','genre_top')]
    X = features.loc[large_b, feature]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    scaler = StandardScaler(copy=False)
    scaler.fit_transform(X_train)
    scaler.transform(X_test)
    SVCrbf = SVC(kernel='rbf')
    SVCrbf.fit(X_train, y_train)
    y_pred = SVCrbf.predict(X_test)
    accuracy_col.append(metrics.accuracy_score(y_test, y_pred).round(3))
#     recall_col.append(metrics.recall_score(y_test, y_pred,average='macro').round(3))
#     f1_col.append(metrics.f1_score(y_test, y_pred,average='macro').round(3))

accuracy_scores['svc'] = accuracy_col
# recall_scores['svc'] = recall_col
# f1_scores['svc'] = f1_col

In [ ]:
accuracy_scores

In [ ]:
accuracy_scores.to_excel("accuracyScores.xlsx")

#### Weighted SVC classifier

In [ ]:
%%time

accuracy_col = []
recall_col = []
f1_col = []

for feature in feature_list:
    y = tracks.loc[large_b, ('track','genre_top')]
    X = features.loc[large_b, feature]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    scaler = StandardScaler(copy=False)
    scaler.fit_transform(X_train)
    scaler.transform(X_test)
    SVCrbf = SVC(kernel='rbf',class_weight={'International' : 4, 'Classical' : 4, 'Instrumental':3,'Folk':2, 'Pop':2}solver='liblinear')
    SVCrbf.fit(X_train, y_train)
    y_pred = SVCrbf.predict(X_test)
    accuracy_col.append(metrics.accuracy_score(y_test, y_pred).round(3))
#     recall_col.append(metrics.recall_score(y_test, y_pred,average='macro').round(3))
#     f1_col.append(metrics.f1_score(y_test, y_pred,average='macro').round(3))

accuracy_scores['weighted svc'] = accuracy_col
# recall_scores['weighted svc'] = recall_col
# f1_scores['weighted svc'] = f1_col

#### SVC optimized with grid search

In [ ]:
%%time

accuracy_col = []
recall_col = []
f1_col = []

for feature in feature_list:
    y = tracks.loc[large_b, ('track','genre_top')]
    X = features.loc[large_b, feature]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    scaler = StandardScaler(copy=False)
    scaler.fit_transform(X_train)
    scaler.transform(X_test)
    SVCrbf = SVC(C=10,kernel='rbf')
    SVCrbf.fit(X_train, y_train)
    y_pred = SVCrbf.predict(X_test)
    accuracy_col.append(metrics.accuracy_score(y_test, y_pred).round(3))
#     recall_col.append(metrics.recall_score(y_test, y_pred,average='macro').round(3))
#     f1_col.append(metrics.f1_score(y_test, y_pred,average='macro').round(3))

accuracy_scores['svc'] = accuracy_col
# recall_scores['svc'] = recall_col
# f1_scores['svc'] = f1_col

In [ ]:
accuracy_scores

#### Optimize SVC with grid search

In [ ]:
# I couldn't fully run this. Took too many hours.

y = tracks.loc[large_b, ('track','genre_top')]
X = features.loc[large_b, 'mfcc']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler(copy=False)
scaler.fit_transform(X_train)
scaler.transform(X_test)


# defining parameter range 
param_grid = {'C': [0.01,0.1, 1, 10, 100],  
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001], 
              'gamma': ['scale', 'auto'],
              'kernel': ['linear', 'poly', 'rbf', 'sigmoid']}  

grid = GridSearchCV(SVC(), param_grid, refit = True, verbose = 3,n_jobs=-1) 

# fitting the model for grid search 
grid.fit(X_train, y_train)

# print best parameter after tuning 
print(grid.best_params_) 
y_pred = grid.predict(X_test) 
   
# print classification report 
print(classification_report(y_test, y_pred))

In [ ]:
accuracy_scores.to_excel("accuracy_scores.xlsx")

# Visualizations

## Confusion Matrix

In [ ]:
# Logistic Regression with all of the features
y = tracks.loc[large_b, ('track','genre_top')]
X = features.loc[large_b, ['chroma_cens','chroma_cqt','chroma_stft','mfcc','rmse','spectral_bandwidth','spectral_centroid',
                           'spectral_contrast','spectral_rolloff','tonnetz','zcr']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler(copy=False)
scaler.fit_transform(X_train)
scaler.transform(X_test)

lr = LogisticRegression(max_iter=1000)
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
print("accuracy score: ", metrics.accuracy_score(y_test, y_pred).round(3))
print("recall score: ", metrics.recall_score(y_test, y_pred,average='macro').round(3))
print("f1 score: ", metrics.f1_score(y_test, y_pred,average='macro').round(3))


In [ ]:
sorted(tracks.loc[large_b,('track','genre_top')].unique())

In [ ]:
confusion = confusion_matrix(y_test, y_pred)
plt.figure(dpi=150)
sns.set(font_scale=0.5)
sns.heatmap(confusion, cmap=plt.cm.Blues, annot=True, square=True, fmt = "d",
           xticklabels=['Classical','Electronic','Experimental','Folk',
                        'Instrumental','International','Pop','Rock'],
                        
           yticklabels=['Classical','Electronic','Experimental','Folk',
                        'Instrumental','International','Pop','Rock'])
plt.xlabel('Predicted Genre')
plt.ylabel('Actual Genre')
plt.title('Confusion Matrix')
plt.yticks(rotation=0);

In [ ]:
# Logistic Regression with weighted genres and all features
y = tracks.loc[large_b, ('track','genre_top')]
X = features.loc[large_b, ['chroma_cens','chroma_cqt','chroma_stft','mfcc','rmse','spectral_bandwidth','spectral_centroid',
                           'spectral_contrast','spectral_rolloff','tonnetz','zcr']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler(copy=False)
scaler.fit_transform(X_train)
scaler.transform(X_test)

lr_weighted = LogisticRegression(class_weight={'International' : 4, 'Classical' : 4, 
                                               'Instrumental':3,'Folk':2, 'Pop':2}, solver='liblinear')
lr_weighted.fit(X_train, y_train)
y_pred = lr_weighted.predict(X_test)
print("weighted lr accuracy score: ", metrics.accuracy_score(y_test, y_pred).round(3))
print("weighted lr recall score: ", metrics.recall_score(y_test, y_pred,average='macro').round(3))
print("weighted lr f1 score: ", metrics.f1_score(y_test, y_pred,average='macro').round(3))


In [ ]:
confusion = confusion_matrix(y_test, y_pred)
plt.figure(dpi=150)
sns.set(font_scale=0.5)
sns.heatmap(confusion, cmap=plt.cm.Blues, annot=True, square=True, fmt = "d",
           xticklabels=['Classical','Electronic','Experimental','Folk',
                        'Instrumental','International','Pop','Rock'],
                        
           yticklabels=['Classical','Electronic','Experimental','Folk',
                        'Instrumental','International','Pop','Rock'])
plt.xlabel('Predicted Genre')
plt.ylabel('Actual Genre')
plt.title('Confusion Matrix')
plt.yticks(rotation=0);

In [ ]:
# Confusion matrix with SVM classifier, all genres
y = tracks.loc[large_b, ('track','genre_top')]
X = features.loc[large_b, ['chroma_cens','chroma_cqt','chroma_stft','mfcc','rmse','spectral_bandwidth','spectral_centroid',
                           'spectral_contrast','spectral_rolloff','tonnetz','zcr']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler(copy=False)
scaler.fit_transform(X_train)
scaler.transform(X_test)
SVCrbf = SVC(kernel='rbf')
SVCrbf.fit(X_train, y_train)
y_pred = SVCrbf.predict(X_test)
print("accuracy score: ", metrics.accuracy_score(y_test, y_pred).round(3))
print("recall score: ", metrics.recall_score(y_test, y_pred,average='macro').round(3))
print("f1 score: ", metrics.f1_score(y_test, y_pred,average='macro').round(3))
print(sklearn.metrics.classification_report(y_test, y_pred))

In [ ]:
confusion = confusion_matrix(y_test, y_pred)
plt.figure(dpi=150)
sns.set(font_scale=0.5)
sns.heatmap(confusion, cmap=plt.cm.Blues, annot=True, square=True, fmt = "d",
           xticklabels=['Classical','Electronic','Experimental','Folk',
                        'Instrumental','International','Pop','Rock'],
                        
           yticklabels=['Classical','Electronic','Experimental','Folk',
                        'Instrumental','International','Pop','Rock'])
plt.xlabel('Predicted Genre')
plt.ylabel('Actual Genre')
plt.title('Confusion Matrix')
plt.yticks(rotation=0)
plt.savefig('svc confusion_matrix'+'.jpg');

## PCA plots

In [ ]:
# Two genres that are RARELY misclassified as each other
# Note: this code was loosely borrowed from the FMA "usage" notebook (https://nbviewer.org/github/mdeff/fma/blob/outputs/usage.ipynb)
# all starting materials found at their github: https://github.com/mdeff/fma

genre1 = tracks['track', 'genre_top'] == 'Rock'
genre2 = tracks['track', 'genre_top'] == 'Classical'

X = features.loc[large_b & (genre1 | genre2), 'mfcc']
X = skl.decomposition.PCA(n_components=2).fit_transform(X)

y = tracks.loc[large_b & (genre1 | genre2), ('track', 'genre_top')]
y = skl.preprocessing.LabelEncoder().fit_transform(y)

plt.figure(figsize=(10,10))
plt.title('PCA Rock vs Classical', fontsize=24)
plt.xlabel('PC1', fontsize=16)
plt.ylabel('PC2', fontsize=16)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.scatter(X[:,0], X[:,1], c=y, cmap='seismic', alpha=0.5, s=2)
X.shape, y.shape


In [ ]:
# Two genres that are RARELY misclassified as each other

genre1 = tracks['track', 'genre_top'] == 'Rock'
genre2 = tracks['track', 'genre_top'] == 'Classical'

X = features.loc[large_b & (genre1 | genre2), 'spectral_contrast']
X = skl.decomposition.PCA(n_components=2).fit_transform(X)

y = tracks.loc[large_b & (genre1 | genre2), ('track', 'genre_top')]
y = skl.preprocessing.LabelEncoder().fit_transform(y)

plt.figure(figsize=(10,10))
plt.title('PCA Rock vs Classical spectral contrast', fontsize=24)
plt.xlabel('PC1', fontsize=16)
plt.ylabel('PC2', fontsize=16)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.scatter(X[:,0], X[:,1], c=y, cmap='seismic', alpha=0.5, s=2)
X.shape, y.shape


In [ ]:
# Two genres that are RARELY misclassified as each other

genre1 = tracks['track', 'genre_top'] == 'Folk'
genre2 = tracks['track', 'genre_top'] == 'Classical'

X = features.loc[large_b & (genre1 | genre2), 'mfcc']
X = skl.decomposition.PCA(n_components=2).fit_transform(X)

y = tracks.loc[large_b & (genre1 | genre2), ('track', 'genre_top')]
y = skl.preprocessing.LabelEncoder().fit_transform(y)

plt.figure(figsize=(10,10))
plt.title('PCA Folk vs Classical', fontsize=24)
plt.xlabel('PC1', fontsize=16)
plt.ylabel('PC2', fontsize=16)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.scatter(X[:,0], X[:,1], c=y, cmap='seismic', alpha=0.5, s=2)
X.shape, y.shape


In [ ]:
# Two genres that are RARELY misclassified as each other

genre1 = tracks['track', 'genre_top'] == 'Rock'
genre2 = tracks['track', 'genre_top'] == 'Classical'

X = features.loc[large_b & (genre1 | genre2), ['chroma_cens','chroma_cqt','chroma_stft','mfcc','rmse','spectral_bandwidth','spectral_centroid',
                     'spectral_contrast','spectral_rolloff','tonnetz','zcr']]
X = skl.decomposition.PCA(n_components=2).fit_transform(X)

y = tracks.loc[large_b & (genre1 | genre2), ('track', 'genre_top')]
y = skl.preprocessing.LabelEncoder().fit_transform(y)

plt.figure(figsize=(10,10))
plt.title('PCA Rock vs Classical all features', fontsize=24)
plt.xlabel('PC1', fontsize=16)
plt.ylabel('PC2', fontsize=16)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

plt.scatter(X[:,0], X[:,1], c=y, cmap='seismic', alpha=0.5, s=2)
X.shape, y.shape


In [ ]:
# Two genres that are RARELY misclassified as each other

genre1 = tracks['track', 'genre_top'] == 'Classical'
genre2 = tracks['track', 'genre_top'] == 'Electronic'

X = features.loc[large_b & (genre1 | genre2), 'mfcc']
X = skl.decomposition.PCA(n_components=2).fit_transform(X)

y = tracks.loc[large_b & (genre1 | genre2), ('track', 'genre_top')]
y = skl.preprocessing.LabelEncoder().fit_transform(y)

plt.figure(figsize=(10,10))
plt.title('PCA Electronic vs Classical', fontsize=24)
plt.xlabel('PC1', fontsize=16)
plt.ylabel('PC2', fontsize=16)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

plt.scatter(X[:,0], X[:,1], c=y, cmap='seismic', alpha=0.5, s=2)
X.shape, y.shape


In [ ]:
# Two genres that are SOMETIMES misclassified as each other

genre1 = tracks['track', 'genre_top'] == 'Electronic'
genre2 = tracks['track', 'genre_top'] == 'Folk'

X = features.loc[large_b & (genre1 | genre2), 'mfcc']
X = skl.decomposition.PCA(n_components=2).fit_transform(X)

y = tracks.loc[large_b & (genre1 | genre2), ('track', 'genre_top')]
y = skl.preprocessing.LabelEncoder().fit_transform(y)

plt.figure(figsize=(10,10))
plt.title('Electronic vs Folk', fontsize=24)
plt.xlabel('PC1', fontsize=16)
plt.ylabel('PC2', fontsize=16)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

plt.scatter(X[:,0], X[:,1], c=y, cmap='RdBu', alpha=0.5, s=2)
X.shape, y.shape


In [ ]:
# Two genres that are OFTEN missclassified as each other

genre1 = tracks['track', 'genre_top'] == 'Rock'
genre2 = tracks['track', 'genre_top'] == 'Electronic'

X = features.loc[large_b & (genre1 | genre2), 'mfcc']
X = skl.decomposition.PCA(n_components=2).fit_transform(X)

y = tracks.loc[large_b & (genre1 | genre2), ('track', 'genre_top')]
y = skl.preprocessing.LabelEncoder().fit_transform(y)

plt.figure(figsize=(10,10))
plt.title('Electronic vs Rock', fontsize=24)
plt.xlabel('PC1', fontsize=16)
plt.ylabel('PC2', fontsize=16)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

plt.scatter(X[:,0], X[:,1], c=y, cmap='seismic', alpha=0.5, s=1)
X.shape, y.shape


In [ ]:
# Two genres that are OFTEN misclassified as each other

genre1 = tracks['track', 'genre_top'] == 'Experimental'
genre2 = tracks['track', 'genre_top'] == 'Electronic'

X = features.loc[large_b & (genre1 | genre2), 'mfcc']
X = skl.decomposition.PCA(n_components=2).fit_transform(X)

y = tracks.loc[large_b & (genre1 | genre2), ('track', 'genre_top')]
y = skl.preprocessing.LabelEncoder().fit_transform(y)

plt.figure(figsize=(10,10))
plt.title('Electronic vs Experimental', fontsize=24)
plt.xlabel('PC1', fontsize=16)
plt.ylabel('PC2', fontsize=16)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

plt.scatter(X[:,0], X[:,1], c=y, cmap='seismic', alpha=0.5, s=1)
X.shape, y.shape


In [ ]:
# Two genres that are OFTEN misclassified as each other

genre1 = tracks['track', 'genre_top'] == 'Pop'
genre2 = tracks['track', 'genre_top'] == 'Experimental'

X = features.loc[large_b & (genre1 | genre2), 'mfcc']
X = skl.decomposition.PCA(n_components=2).fit_transform(X)

y = tracks.loc[large_b & (genre1 | genre2), ('track', 'genre_top')]
y = skl.preprocessing.LabelEncoder().fit_transform(y)

plt.figure(figsize=(10,10))
plt.title('Pop vs Experimental', fontsize=24)
plt.xlabel('PC1', fontsize=16)
plt.ylabel('PC2', fontsize=16)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

plt.scatter(X[:,0], X[:,1], c=y, cmap='RdBu', alpha=0.5, s=1)
X.shape, y.shape


In [ ]:
# Two genres that are OFTEN misclassified as each other

genre1 = tracks['track', 'genre_top'] == 'Pop'
genre2 = tracks['track', 'genre_top'] == 'Electronic'

X = features.loc[large_b & (genre1 | genre2), 'mfcc']
X = skl.decomposition.PCA(n_components=2).fit_transform(X)

y = tracks.loc[large_b & (genre1 | genre2), ('track', 'genre_top')]
y = skl.preprocessing.LabelEncoder().fit_transform(y)

plt.figure(figsize=(10,10))
plt.title('Pop vs Electronic', fontsize=24)
plt.xlabel('PC1', fontsize=16)
plt.ylabel('PC2', fontsize=16)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

plt.scatter(X[:,0], X[:,1], c=y, cmap='RdBu', alpha=0.5, s=1)
X.shape, y.shape
